In [ ]:
'''

    create a dynamodb client
    list tables in dynamodb
    create table in dynamodb
    list item in table
    insert two item to dynamodb
    list item in table
    get item from table
    remove item from table
    query
    scan

What kind difference of query and scan?

參考連結
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html

DynamoDB 對應Python 資料型態
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/customizations/dynamodb.html#ref-valid-dynamodb-types

'''

In [ ]:
!pip install boto3 awscli

In [ ]:
# create a dynamodb client

import boto3

# Get the service resource.
dynamodb = boto3.resource('dynamodb', endpoint_url='http://localstack-main:4566')

In [ ]:
# list tables in dynamodb

list(dynamodb.tables.all())

In [ ]:
# create table in dynamodb

table = dynamodb.create_table(
    TableName='s3-objects-list', 
    KeySchema=[
        {
            'AttributeName': 'image_name',
            'KeyType': 'HASH'  # Partition key
        },
        {
            'AttributeName': 'upload_time',
            'KeyType': 'RANGE'  # Sort key
        }
    ], 
    AttributeDefinitions=[
        {
            'AttributeName': 'image_name',
            'AttributeType': 'S'  # N 代表數字； S 代表字串。
        },
        {
            'AttributeName': 'upload_time',
            'AttributeType': 'S'  
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)



In [ ]:
# list tables in dynamodb

list(dynamodb.tables.all())

In [ ]:
# list item in table

table = dynamodb.Table('s3-objects-list')

print(table.item_count)

In [ ]:
# insert two item to dynamodb

# Float types are not supported. Use Decimal types instead.

from decimal import Decimal
import datetime

now1 = str(datetime.datetime.now())
print(now1)

table.put_item(
   Item={
        "image_name": 'first_image',
        "upload_time": now1,
        "image_url": 'https://s3_url/first_image'
    }
)

now2 = str(datetime.datetime.now())
print(now2)

table.put_item(
   Item={
        "image_name": 'second_image',
        "upload_time": now2,
        "image_url": 'https://s3_url/second_image'
    }
)

In [ ]:
# list item in table

print(table.item_count)

In [ ]:
# get item from table

from pprint import pprint

response = table.get_item(
    Key={
        'image_name': 'second_image',
        'upload_time': now2
    }
)

pprint(response)

item = response['Item']
pprint(item)

In [ ]:
# remove item from table

# For the primary key, you must provide all of the attributes. 
#     For example, with a simple primary key, 
#         you only need to provide a value for the partition key. 
#     For a composite primary key, 
#         you must provide values for both the partition key and the sort key.


table.delete_item(
    Key={
        "image_name": 'second_image',
        "upload_time": now2,
    }
)

In [ ]:
# query

# The Query operation finds items based on primary key values. 

# To add conditions to scanning and querying the table, 
#     you will need to import the boto3.dynamodb.conditions.Key 
#     and boto3.dynamodb.conditions.Attr classes.

from boto3.dynamodb.conditions import Key, Attr
from pprint import pprint

response = table.query(
    KeyConditionExpression=Key('image_name').eq('first_image')
)

items = response['Items']
pprint(items)

In [ ]:
# scan

# The Scan operation returns one or more items and item attributes 
#     by accessing every item in a table or a secondary index.

# To add conditions to scanning and querying the table, 
#     you will need to import the boto3.dynamodb.conditions.Key 
#     and boto3.dynamodb.conditions.Attr classes.

# If the total number of scanned items exceeds the maximum dataset size limit of 1 MB, 
#     the scan stops and results are returned to the user as a LastEvaluatedKey value 
#     to continue the scan in a subsequent operation.

from boto3.dynamodb.conditions import Key, Attr
from pprint import pprint

response = table.scan(
    FilterExpression=Attr('image_url').contains('s3_url')
)

items = response['Items']
pprint(items)